In [1]:
import sys
sys.path.append('\\\\shared.ad.syr.edu\\AS-Filer\\PHY-PRL-labshare\\Programs\\python\\Drivers')

In [2]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
from AgilentN5230A import AgilentN5230A

In [4]:
pna = AgilentN5230A(16)
pna.connect()
pna.setupMeas()
pna.avgCount(50)

Initialized S21 Measurement 'S21Meas'


In [5]:
savepath = 'Z:\\Experiments\\Vortex_measurements\\Kenneth\\6ResNb122216\\'

In [13]:
f0s = [3.912897,4.79087,6.767286,7.70971]
spans = [0.025,0.52,0.15,3.78]
pows = [0,-5,-10,-15,-20,-25,-30,-35,-40,-45,-50,-55,-60,-65]

for (f0,span) in zip(f0s,spans):
    pna.centerF(f0)
    pna.spanf(span)
    for pow in pows:
        pna.pwr(pow)
        pna.avgCount((abs(pow)+1)*15)
        freq,S21 = pna.getData(True)
        S21_real, S21_imag = np.real(S21), np.imag(S21)
        S21_mag, S21_phase = np.sqrt(S21_real**2 + S21_imag**2), np.angle(S21, deg=True) # phase in degree
        S21_mag_log = 20.0 * np.log10(S21_mag)
        savetxt(savepath+'Res'+str(round(f0))[0]+'P'+str(abs(pow))+'.txt',list(zip(freq,S21_mag_log,S21_phase)),delimiter = '\t',newline = '\r\n',fmt='%.6e')
    
#pna.centerF(2.931344)
#pna.spanf(5)
#pna.numPoints(1601)
#freq,S21 = pna.getData(True)
#S21_real, S21_imag = np.real(S21), np.imag(S21)
#S21_mag, S21_phase = np.sqrt(S21_real**2 + S21_imag**2), np.angle(S21, deg=True) # phase in degree
#S21_mag_log = 20.0 * np.log10(S21_mag)
#savetxt(savepath+'Res'+str()+'P'+str()+'.txt',list(zip(freq,S21_mag_log,S21_phase)),delimiter = '\t',newline = '\r\n',fmt='%.6e')

In [17]:
####################################
freqs = [1.971018e9,2.931344e9,3.898026e9,4.765892e9,6.739851e9,7.638164]
powss = [0,-10,-20,-30,-40,-50,-60,-70,-80,-90]
points = [600,100,100]
spans = [4e6,4e6,4e6,4e6,150e6]
avgs = [50,51,100,101,200,400,5000,8000,20001,20000]
span_bg = 8.0e6
IFbandwidth = 1e3
pna.setIF(IFbandwidth)

save_path = 'Z:\Experiments\Vortex_measurements\Kenneth\6ResNb122016'
surfix = ''
####################################
background_control = [0,1,2]

# dictionary for parameters to be saved in json file. Solely for log.
xID, xStart, xEnd, xStep = 'Power (dBm)', startPwr, endPwr, stepPwr
yID, yStart, yEnd, yStep = 'Frequency (Hz)',startFreq, endFreq, (endFreq-startFreq)/(numPoints-1)

# plot setup
plotobj= myplots.plot1D_2sub(xlabel1 = '', ylabel1='S21 (dB)', 
                             xlabel2 = 'Freq. (Hz)', ylabel2='Phase(degree)',
                             title = 'S21 vs Freq.', plotStyle='b-o')

# preconfigure instrument
pna.setupMeas()
pna.pwrOn()
pna.setSweepType('linear')

# data initialization
S21_mag_log_sum, S21_phase_sum = np.array([]), np.array([])
S21_mag_log_avg, S21_phase_avg = np.array([]), np.array([])

for fnum in range(len(freqs)):
    for pnum in range(len(powss)):
        freqC = freq = freqs[fnum]
        for bgc in background_control:
            avg = avgs[pnum]
            for avgidx in range(avg):
                pna.avgClear()
                if bgc == 0:
                    freq = freqs[fnum]
                    span = spans[fnum]

                if bgc == 1:
                    freq = freqs[fnum]-3e6
                    span = 2e6

                if bgc ==2:
                    freq = freqs[fnum]+3e6
                    span = 2e6
                point = points[bgc]
                pows = powss[pnum]

                print("Mesuring Freq {}: Span {}: Pow: {} AVG: {} PTS: {}".format(freq, span, pows, avg, point))
    ###############################################
                # dictionary for parameters to be saved in json file. Solely for log.
                header = '{:s} {:s} {:s}'.format('Frequency(Hz)', 'mag(dB)', 'phase(degree)')
                para = dict([('readout Freq (Hz)',freqC),('background_control',bgc),('readout Power (dBm)', pows),('numPoints',point),
                             ('IFbandwidth (Hz)', IFbandwidth),('avg',(avgidx+1)),('header', header)])
                # file setup
                FAA = float(lsci.tempGet('B')) # unicode to num  
                baseStr = 'S21vsV_fr{:.9g}_BGc{:.9g}_Pr{:.9g}_T{:.3g}_{}'
                baseFileName = baseStr.format(freqC/1e9,bgc, pows, FAA, surfix)
                dataFileName  = os.path.join(save_path, baseFileName + '.dat')
                logFileName = os.path.join(save_path, baseFileName + '.json')
                with open(logFileName, 'w') as f:
                    json.dump(para,f,indent=4, sort_keys=True)

                f_data = open(dataFileName, 'a')
###############################################################
                pna.centerF(freq)
                pna.spanf(span)
                pna.pwr(pows)
                pna.numPoints(point)
##############################################################
                freq,S21 = pna.getData(cplx=True)

                # S21 conversion
                S21_real, S21_imag = np.real(S21), np.imag(S21)
                S21_mag, S21_phase = np.sqrt(S21_real**2 + S21_imag**2), np.angle(S21, deg=True) # phase in degree
                S21_mag_log = 20.0 * np.log10(S21_mag)    # dB
                
                if S21_mag_log_sum.size == 0: # when S21_mag_2D is empty
                    S21_mag_log_sum = S21_mag_log
                    S21_mag_log_sum.shape = (S21_mag_log.size, 1)
                    S21_phase_sum = S21_phase
                    S21_phase_sum.shape = (S21_phase.size, 1)
                else:
                    S21_mag_log_sum = S21_mag_log_sum + S21_mag_log
                    S21_phase_sum = S21_phase_sum + S21_phase
                
                S21_mag_log_avg = S21_mag_log_sum/(avgidx+1)
                S21_phase_avg = S21_phase_sum/(avgidx+1)
                
                # plot and update
                plotobj.update(freq, S21_mag_log_avg, freq, S21_phase_avg)
                
                # save data in each iteration
                string = '{:.9g}\t{:.3g}\t{:.3g} \n'.format(sweeper.curr_val, S21_mag_log_avg,S21_phase_avg)
                f_data.write(string)

                # update sweep
                sweeper.update()
                
#################################################                
############## sweep end #########################    

# Post measurement    
pna.pwrOff()
f_data.close()
f.close()
print('Measurement done!!')

In [ ]:
                freq_r = round(freq,3)
                span_r = round(span,1)
                freq_str = str(freq_r)
                span_str = str (span_r)
                pow_str = str(pows)
                point_str = str(point)
                avg_str = str(avg)
                num_str = str(num)


                head = [0.000,freq_r,0.000,0.000,pows,0.000,0.000,0.000,point,avg]
                f = open("head.txt","w")
                i = 0
                for h in head:
                    out = str(head[i])+"\n"
                    f.write(out)
                    i +=1
                f.close()


                output = "output.txt"
                print("Writing {} to file\n".format(output))
                savetxt(output,np.transpose([x_mag,y_mag_log,y_phase,y_mag,y_real,y_imag]),delimiter=",")

                filename = ["head.txt",output]
                name = "Niobium05122016_"+freq_str+"_"+span_str+"_"+pow_str+"_"+point_str+"_"+avg_str+"_"+num_str+".txt"
                with open(name,"w") as outfile:
                    for fname in filename:
                        with open(fname) as infile:
                            for line in infile:
                                outfile.write(line)

In [39]:
savepath+'save.txt'

'Z:\\Experiments\\Vortex_measurements\\Kenneth\\6ResNb122016\\save.txt'

In [12]:
str(round(3,2))[0]

'3'

In [44]:
str(round(2.24612346))

'2'

In [55]:
pna.centerF(7.638164)
pna.spanf(0.25)
pna.numPoints(1601)